In [2]:
import re
import pandas as pd
import numpy as np
data = pd.read_csv("../data400_share/beer.csv")

In [8]:
data.head()

,index,beer/ABV,beer/beerId,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,review/timeStruct,review/timeUnix,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName
0,40163,5.0,46634,14338,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,"{'min': 38, 'hour': 3, 'mday': 16, 'sec': 10, ...",1229398690,NaN,NaN,NaN,NaN,RblWthACoz
1,8135,11.0,3003,395,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,"{'min': 38, 'hour': 23, 'mday': 8, 'sec': 58, ...",1218238738,NaN,NaN,NaN,NaN,BeerSox
2,10529,4.7,961,365,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,"{'min': 7, 'hour': 18, 'mday': 26, 'sec': 2, '...",1101492422,NaN,NaN,NaN,Male,mschofield
3,44610,4.4,429,1,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,"{'min': 7, 'hour': 1, 'mday': 20, 'sec': 5, 'y...",1308532025,1.209827e+09,"Aug 10, 1976",208508400.0,Male,molegar76
4,37062,4.4,4904,1417,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,"{'min': 51, 'hour': 6, 'mday': 12, 'sec': 48, ...",1299912708,NaN,NaN,NaN,NaN,Brewbro000


In [9]:
data.columns

Index(['index', 'beer/ABV', 'beer/beerId', 'beer/brewerId', 'beer/name',
       'beer/style', 'review/appearance', 'review/aroma', 'review/overall',
       'review/palate', 'review/taste', 'review/text', 'review/timeStruct',
       'review/timeUnix', 'user/ageInSeconds', 'user/birthdayRaw',
       'user/birthdayUnix', 'user/gender', 'user/profileName'],
      dtype='object')

In [6]:
len(data)

37500

In [15]:
data["review/timeStruct"][5]

"{'min': 16, 'hour': 1, 'mday': 20, 'sec': 9, 'year': 2011, 'wday': 1, 'mon': 12, 'isdst': 0, 'yday': 354}"

In [29]:
data["age_years"]=np.floor(data['user/ageInSeconds']/31536000)
data[["age_years"]].head()

,age_years
0,NaN
1,NaN
2,NaN
3,38.0
4,NaN


In [11]:
len(data["beer/name"].unique())

1688

In [7]:
len(data["user/profileName"].unique())

7442

In [8]:
train = data

In [9]:
train['word_count'] = train['review/text'].apply(lambda x: len(str(x).split(" ")))
train[['review/text','word_count']].head()


,review/text,word_count
0,Pours a clouded gold with a thin white head. N...,94
1,12oz bottle into 8oz snifter.\t\tDeep ruby red...,99
2,First enjoyed at the brewpub about 2 years ago...,63
3,First thing I noticed after pouring from green...,67
4,A: pours an amber with a one finger head but o...,90


In [10]:
train['char_count'] = train['review/text'].str.len() ## this also includes spaces
train[['review/text','char_count']].head()

,review/text,char_count
0,Pours a clouded gold with a thin white head. N...,519.0
1,12oz bottle into 8oz snifter.\t\tDeep ruby red...,568.0
2,First enjoyed at the brewpub about 2 years ago...,392.0
3,First thing I noticed after pouring from green...,391.0
4,A: pours an amber with a one finger head but o...,486.0


In [11]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

train['avg_word_len'] = train['review/text'].apply(lambda x: avg_word(str(x)))
train[['review/text','avg_word_len']].head()


,review/text,avg_word_len
0,Pours a clouded gold with a thin white head. N...,4.531915
1,12oz bottle into 8oz snifter.\t\tDeep ruby red...,4.529412
2,First enjoyed at the brewpub about 2 years ago...,4.865672
3,First thing I noticed after pouring from green...,4.820896
4,A: pours an amber with a one finger head but o...,4.236559


In [12]:
stop = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", 
        "she", "her", "hers", "herself", "it", "its", "itself", "they", 
        "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", 
        "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", 
        "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", 
        "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", 
        "for", "with", "about", "against", "between", "into", "through", "during", "before", 
        "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", 
        "over", "under", "again", "further", "then", "once", "here", "there", "when", 
        "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", 
        "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", 
        "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

train['stopwords'] = train['review/text'].apply(lambda x: len([x for x in str(x).split() if x in stop]))
train[['review/text','stopwords']].head()

,review/text,stopwords
0,Pours a clouded gold with a thin white head. N...,36
1,12oz bottle into 8oz snifter.\t\tDeep ruby red...,35
2,First enjoyed at the brewpub about 2 years ago...,18
3,First thing I noticed after pouring from green...,22
4,A: pours an amber with a one finger head but o...,38


In [13]:
train['questions'] = train['review/text'].apply(lambda x: len([x for x in str(x).split() if '?' in str(x)]))
train[['review/text','questions']].head()

,review/text,questions
0,Pours a clouded gold with a thin white head. N...,1
1,12oz bottle into 8oz snifter.\t\tDeep ruby red...,0
2,First enjoyed at the brewpub about 2 years ago...,0
3,First thing I noticed after pouring from green...,0
4,A: pours an amber with a one finger head but o...,0


In [14]:
train['ellipses'] = train['review/text'].apply(lambda x: len([x for x in str(x).split() if '...' in str(x)]))
train[['review/text','ellipses']].head()

,review/text,ellipses
0,Pours a clouded gold with a thin white head. N...,0
1,12oz bottle into 8oz snifter.\t\tDeep ruby red...,0
2,First enjoyed at the brewpub about 2 years ago...,0
3,First thing I noticed after pouring from green...,0
4,A: pours an amber with a one finger head but o...,0


In [15]:
train['exclamations'] = train['review/text'].apply(lambda x: len([x for x in str(x).split() if '!' in str(x)]))
train[['review/text','exclamations']].head()

,review/text,exclamations
0,Pours a clouded gold with a thin white head. N...,0
1,12oz bottle into 8oz snifter.\t\tDeep ruby red...,0
2,First enjoyed at the brewpub about 2 years ago...,0
3,First thing I noticed after pouring from green...,0
4,A: pours an amber with a one finger head but o...,0


In [16]:
train['plus'] = train['review/text'].apply(lambda x: len([x for x in str(x).split() if '+' in str(x)]))
train[['review/text','plus']].head()

,review/text,plus
0,Pours a clouded gold with a thin white head. N...,0
1,12oz bottle into 8oz snifter.\t\tDeep ruby red...,0
2,First enjoyed at the brewpub about 2 years ago...,0
3,First thing I noticed after pouring from green...,0
4,A: pours an amber with a one finger head but o...,0


In [17]:
train['star'] = train['review/text'].apply(lambda x: len([x for x in str(x).split() if '*' in str(x)]))
train[['review/text','star']].head()

,review/text,star
0,Pours a clouded gold with a thin white head. N...,0
1,12oz bottle into 8oz snifter.\t\tDeep ruby red...,0
2,First enjoyed at the brewpub about 2 years ago...,0
3,First thing I noticed after pouring from green...,0
4,A: pours an amber with a one finger head but o...,0


In [18]:
train['numerics'] = train['review/text'].apply(lambda x: len([x for x in str(x).split() if x.isdigit()]))
train[['review/text','numerics']].head()

,review/text,numerics
0,Pours a clouded gold with a thin white head. N...,0
1,12oz bottle into 8oz snifter.\t\tDeep ruby red...,0
2,First enjoyed at the brewpub about 2 years ago...,1
3,First thing I noticed after pouring from green...,0
4,A: pours an amber with a one finger head but o...,0


In [19]:
train['review/text'][0]

'Pours a clouded gold with a thin white head. Nose is quite floral with a larger amount of spices added. Definitely a spice forward fragrance. Flavor has an odd burn that hits on the first sip. After it fades it seems like a dirty vanilla aftertaste. Perhaps this is the absinthe? Regardless of that, I get a quite spiced tone on the tongue. Almost feel a little heat from it. I think that my inexperienced palate on these spices is contributing to my ignorance of what precisely they are. Overall a nice drinker indeed.'

In [20]:
train['upper'] = train['review/text'].str.findall(r'[A-Z]').str.len()
train[['review/text','upper']].head()

,review/text,upper
0,Pours a clouded gold with a thin white head. N...,11.0
1,12oz bottle into 8oz snifter.\t\tDeep ruby red...,10.0
2,First enjoyed at the brewpub about 2 years ago...,7.0
3,First thing I noticed after pouring from green...,11.0
4,A: pours an amber with a one finger head but o...,9.0


In [21]:
train['review/text'] = train['review/text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
train['review/text'].head()

0    pours a clouded gold with a thin white head. n...
1    12oz bottle into 8oz snifter. deep ruby red hu...
2    first enjoyed at the brewpub about 2 years ago...
3    first thing i noticed after pouring from green...
4    a: pours an amber with a one finger head but o...
Name: review/text, dtype: object

In [22]:
train['review/text'] = train['review/text'].str.replace('[^\w\s]','')
train['review/text'].head()

0    pours a clouded gold with a thin white head no...
1    12oz bottle into 8oz snifter deep ruby red hue...
2    first enjoyed at the brewpub about 2 years ago...
3    first thing i noticed after pouring from green...
4    a pours an amber with a one finger head but on...
Name: review/text, dtype: object

In [23]:
train['review/text'] = train['review/text'].apply(lambda x: " ".join(x for x in str(x).split() if x not in stop))
train['review/text'].head()

0    pours clouded gold thin white head nose quite ...
1    12oz bottle 8oz snifter deep ruby red hue one ...
2    first enjoyed brewpub 2 years ago finally mana...
3    first thing noticed pouring green bottle glass...
4    pours amber one finger head strong pour head r...
Name: review/text, dtype: object

In [24]:
freq = pd.Series(' '.join(train['review/text']).split()).value_counts()[:10]
freq

beer     46502
head     36240
taste    26333
nice     24451
good     22929
malt     21940
like     21329
hops     20973
one      20593
sweet    20123
dtype: int64

In [25]:
freq = list(freq.index) #take out most popular words
train['review/text'] = train['review/text'].apply(lambda x: " ".join(x for x in str(x).split() if x not in freq))
train['review/text'].head()

0    pours clouded gold thin white nose quite flora...
1    12oz bottle 8oz snifter deep ruby red hue fing...
2    first enjoyed brewpub 2 years ago finally mana...
3    first thing noticed pouring green bottle glass...
4    pours amber finger strong pour recedes light l...
Name: review/text, dtype: object

In [26]:
freq = pd.Series(' '.join(train['review/text']).split()).value_counts()[-10:]
freq

rags                 1
chocobut             1
idrinkgas            1
513                  1
pickiest             1
metallicminerally    1
harringtons          1
serveral             1
aerican              1
turbocharged         1
dtype: int64

In [27]:
freq = list(freq.index)
train['review/text'] = train['review/text'].apply(lambda x: " ".join(x for x in str(x).split() if x not in freq))
train['review/text'].head() #remove least common words

0    pours clouded gold thin white nose quite flora...
1    12oz bottle 8oz snifter deep ruby red hue fing...
2    first enjoyed brewpub 2 years ago finally mana...
3    first thing noticed pouring green bottle glass...
4    pours amber finger strong pour recedes light l...
Name: review/text, dtype: object

In [28]:
train.head()

,index,beer/ABV,beer/beerId,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,...,char_count,avg_word_len,stopwords,questions,ellipses,exclamations,plus,star,numerics,upper
0,40163,5.0,46634,14338,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,...,519.0,4.531915,36,1,0,0,0,0,0,11.0
1,8135,11.0,3003,395,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,...,568.0,4.529412,35,0,0,0,0,0,0,10.0
2,10529,4.7,961,365,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,...,392.0,4.865672,18,0,0,0,0,0,1,7.0
3,44610,4.4,429,1,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,...,391.0,4.820896,22,0,0,0,0,0,0,11.0
4,37062,4.4,4904,1417,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,...,486.0,4.236559,38,0,0,0,0,0,0,9.0


In [42]:
import textblob
import nltk
#nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [43]:
train.columns

Index(['index', 'beer/ABV', 'beer/beerId', 'beer/brewerId', 'beer/name',
       'beer/style', 'review/appearance', 'review/aroma', 'review/overall',
       'review/palate', 'review/taste', 'review/text', 'review/timeStruct',
       'review/timeUnix', 'user/ageInSeconds', 'user/birthdayRaw',
       'user/birthdayUnix', 'user/gender', 'user/profileName', 'word_count',
       'char_count', 'avg_word_len', 'stopwords', 'questions', 'ellipses',
       'exclamations', 'plus', 'star', 'numerics', 'upper'],
      dtype='object')

In [44]:
from textblob import Word
train['review/text'] = train['review/text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in str(x).split()]))
train['review/text'].head()

0    pours clouded gold thin white nose quite flora...
1    12oz bottle 8oz snifter deep ruby red hue fing...
2    first enjoyed brewpub 2 year ago finally manag...
3    first thing noticed pouring green bottle glass...
4    pours amber finger strong pour recedes light l...
Name: review/text, dtype: object

In [37]:
#train['bigrams'] = train['review/text'].apply(lambda x: TextBlob(x).ngrams(2))

In [38]:
#train[['review/text','bigrams']].head()

In [39]:
sum(train["plus"])

503

In [40]:
sum(train["exclamations"]) #definitely useful

11295

In [41]:
sum(train["star"])

391

In [7]:
open_beer = pd.read_csv("open-beer-database.csv")

ParserError: Error tokenizing data. C error: Expected 2 fields in line 94, saw 3
